In [1]:
%matplotlib inline

import matplotlib.pyplot as plt
from dask.dot import dot_graph
import itertools
import logging
import netCDF4
import numpy
import dask.array as da
from dask import delayed
import time
from dask.distributed import Client
from urllib import request


client = Client('scheduler:8786')
# client = Client(processes=False)
client.ncores()

download_location = '/temp'
data_url = 'http://172.22.0.1:8910'
max_download_attempts = 5

all_models = ['ACCESS1-0', 'BNU-ESM', 'CCSM4', 'CESM1-BGC', 'CNRM-CM5', 'CSIRO-Mk3-6-0', 'CanESM2', 'GFDL-CM3', 'GFDL-ESM2G', 'GFDL-ESM2M', 'IPSL-CM5A-LR', 'IPSL-CM5A-MR', 'MIROC-ESM-CHEM', 'MIROC-ESM', 'MIROC5', 'MPI-ESM-LR', 'MPI-ESM-MR', 'MRI-CGCM3', 'NorESM1-M', 'bcc-csm1-1', 'inmcm4']
all_models = ['ACCESS1-0'] 
all_vars = ['tasmax', 'pr']
all_years = {
     'historical': list(range(1971, 1976))
}

def get_dataset_url(variable, scenario, model, year, prefix = data_url):
    prefix_filename = '/'.join([prefix, 'NEX-GDDP', 'BCSD', scenario, 'day', 'atmos', variable, 'r1i1p1', 'v1.0'])
    # prefix_filename = data_url + '/data'
    filename = '_'.join([variable, 'day', 'BCSD', scenario, 'r1i1p1', model, str(year) + '.nc'])
    url = '/'.join([prefix_filename, filename])
    return url

def get_context(year, **kwargs):
    variables = [kwargs.get('variable')] if kwargs.get('variable') else all_vars
    scenarios = ['historical']
    models = [kwargs.get('model')] if kwargs.get('model') else all_models
    outlist = []
    combinations = list(itertools.product(variables, scenarios, models))
    result = list(map(lambda comb: [ *comb, year ], combinations))
    return result

def get_year_ensemble(year, variable = 'tasmax'):
    context = get_context(year, variable = variable)
    datasets = list(map(lambda x: str(get_dataset_url(*x)), context))
    return datasets

def to_dataset(filename):
    return netCDF4.Dataset(filename)

def download_file(url):
    print("url: " + url)
    attempts = 0
    success = False
    filename = ""
    while attempts < max_download_attempts and not success:
        time.sleep(2 ** attempts)
        filename = '/'.join([download_location, str(url.split('/')[-1])])
        print("Downloading file at " + filename)
        u = request.urlopen(url)
        f = open(filename, 'wb')
        f.write(u.read())
        f.close()
        success = True
        break
    return filename

In [2]:
client.ncores()

{'tcp://172.22.0.4:41545': 8}

In [3]:
datasets_tasmax = list(map(get_year_ensemble, all_years['historical']))
datasets_pr = list(map(lambda x : get_year_ensemble(x, variable = 'pr'), all_years['historical']))
datasets_tasmax

[['http://172.22.0.1:8910/NEX-GDDP/BCSD/historical/day/atmos/tasmax/r1i1p1/v1.0/tasmax_day_BCSD_historical_r1i1p1_ACCESS1-0_1971.nc'],
 ['http://172.22.0.1:8910/NEX-GDDP/BCSD/historical/day/atmos/tasmax/r1i1p1/v1.0/tasmax_day_BCSD_historical_r1i1p1_ACCESS1-0_1972.nc'],
 ['http://172.22.0.1:8910/NEX-GDDP/BCSD/historical/day/atmos/tasmax/r1i1p1/v1.0/tasmax_day_BCSD_historical_r1i1p1_ACCESS1-0_1973.nc'],
 ['http://172.22.0.1:8910/NEX-GDDP/BCSD/historical/day/atmos/tasmax/r1i1p1/v1.0/tasmax_day_BCSD_historical_r1i1p1_ACCESS1-0_1974.nc'],
 ['http://172.22.0.1:8910/NEX-GDDP/BCSD/historical/day/atmos/tasmax/r1i1p1/v1.0/tasmax_day_BCSD_historical_r1i1p1_ACCESS1-0_1975.nc']]

In [4]:
dsets = []
da_dsets = []
for row in datasets_tasmax:
    print("Processing year")
    fnames = list(map(download_file, row))
    datasets_year = list(map(lambda dset: netCDF4.Dataset(dset), fnames))
    dsets.append(datasets_year)
    
    dask_dsets_year = map(lambda dset: da.from_array(dset['tasmax'], chunks=(366, 120, 120)), datasets_year)
    da_dsets.append(list(dask_dsets_year))

da_dsets

Processing year
url: http://172.22.0.1:8910/NEX-GDDP/BCSD/historical/day/atmos/tasmax/r1i1p1/v1.0/tasmax_day_BCSD_historical_r1i1p1_ACCESS1-0_1971.nc
Processing year
url: http://172.22.0.1:8910/NEX-GDDP/BCSD/historical/day/atmos/tasmax/r1i1p1/v1.0/tasmax_day_BCSD_historical_r1i1p1_ACCESS1-0_1972.nc
Processing year
url: http://172.22.0.1:8910/NEX-GDDP/BCSD/historical/day/atmos/tasmax/r1i1p1/v1.0/tasmax_day_BCSD_historical_r1i1p1_ACCESS1-0_1973.nc
Processing year
url: http://172.22.0.1:8910/NEX-GDDP/BCSD/historical/day/atmos/tasmax/r1i1p1/v1.0/tasmax_day_BCSD_historical_r1i1p1_ACCESS1-0_1974.nc
Processing year
url: http://172.22.0.1:8910/NEX-GDDP/BCSD/historical/day/atmos/tasmax/r1i1p1/v1.0/tasmax_day_BCSD_historical_r1i1p1_ACCESS1-0_1975.nc


[[dask.array<array, shape=(365, 720, 1440), dtype=float32, chunksize=(365, 120, 120)>],
 [dask.array<array, shape=(366, 720, 1440), dtype=float32, chunksize=(366, 120, 120)>],
 [dask.array<array, shape=(365, 720, 1440), dtype=float32, chunksize=(365, 120, 120)>],
 [dask.array<array, shape=(365, 720, 1440), dtype=float32, chunksize=(365, 120, 120)>],
 [dask.array<array, shape=(365, 720, 1440), dtype=float32, chunksize=(365, 120, 120)>]]

In [5]:
da_dsets = [row[0] for row in da_dsets]
da_dsets[0]

dask.array<array, shape=(365, 720, 1440), dtype=float32, chunksize=(365, 120, 120)>

In [6]:
same_length_dsets = list(filter(lambda dset: dset.chunks[0][0] == 365, da_dsets))
same_length_dsets

[dask.array<array, shape=(365, 720, 1440), dtype=float32, chunksize=(365, 120, 120)>,
 dask.array<array, shape=(365, 720, 1440), dtype=float32, chunksize=(365, 120, 120)>,
 dask.array<array, shape=(365, 720, 1440), dtype=float32, chunksize=(365, 120, 120)>,
 dask.array<array, shape=(365, 720, 1440), dtype=float32, chunksize=(365, 120, 120)>]

In [7]:
# To stack the datasets along a new axis:
avg_stack = da.stack(same_length_dsets, axis = 0)

@delayed
def custom_average(a, **kwargs):
    return numpy.mean(a, **kwargs)

model_averages = custom_average(avg_stack, axis = 0)
model_averages

Delayed('custom_average-32549568-e72e-4393-8157-0cd10e35376a')

In [8]:
model_averages.compute()

array([[[ 273.55682373,  273.58920288,  273.59063721, ...,  273.59851074,
          273.56674194,  273.54931641],
        [ 273.79293823,  273.82687378,  273.84289551, ...,  273.77865601,
          273.77618408,  273.7746582 ],
        [ 273.99713135,  274.03924561,  274.05969238, ...,  273.9553833 ,
          273.95996094,  273.97366333],
        ..., 
        [ 236.60502625,  236.74078369,  236.84527588, ...,  236.4808197 ,
          236.48919678,  236.52020264],
        [ 236.52481079,  236.66943359,  236.77598572, ...,  236.43704224,
          236.44009399,  236.46292114],
        [ 236.47286987,  236.58821106,  236.65655518, ...,  236.41146851,
          236.41757202,  236.40863037]],

       [[ 273.24267578,  273.27508545,  273.3052063 , ...,  273.32641602,
          273.29885864,  273.25170898],
        [ 273.65966797,  273.69671631,  273.71954346, ...,  273.70153809,
          273.67755127,  273.64920044],
        [ 274.02685547,  274.07055664,  274.09963989, ...,  274.03289795

In [11]:
# To concatenate dask arrays
stack = da.concatenate(da_dsets, axis = 0)
stack_rechunked = stack.rechunk((731, 120, 120))

stack_rechunked

dask.array<rechunk-merge, shape=(1826, 720, 1440), dtype=float32, chunksize=(731, 120, 120)>

In [12]:
@delayed
def custom_percentile(a, q = 99, **kwargs):
    return numpy.percentile(a, q, **kwargs)

@delayed
def custom_nanpercentile(a, q = 99, **kwargs):
    return numpy.nanpercentile(a, q, **kwargs)


# custom_percentile(da.from_array(numpy.arange(1000000).reshape(100, 100, 100), chunks=(10))).compute()
custom_nanpercentile(da.from_array(numpy.arange(1000000).reshape(100, 100, 100), chunks=(10)), axis=2).compute()

array([[  9.80100000e+01,   1.98010000e+02,   2.98010000e+02, ...,
          9.79801000e+03,   9.89801000e+03,   9.99801000e+03],
       [  1.00980100e+04,   1.01980100e+04,   1.02980100e+04, ...,
          1.97980100e+04,   1.98980100e+04,   1.99980100e+04],
       [  2.00980100e+04,   2.01980100e+04,   2.02980100e+04, ...,
          2.97980100e+04,   2.98980100e+04,   2.99980100e+04],
       ..., 
       [  9.70098010e+05,   9.70198010e+05,   9.70298010e+05, ...,
          9.79798010e+05,   9.79898010e+05,   9.79998010e+05],
       [  9.80098010e+05,   9.80198010e+05,   9.80298010e+05, ...,
          9.89798010e+05,   9.89898010e+05,   9.89998010e+05],
       [  9.90098010e+05,   9.90198010e+05,   9.90298010e+05, ...,
          9.99798010e+05,   9.99898010e+05,   9.99998010e+05]])

In [13]:
# Running some speed tests
op = custom_percentile(stack, axis = 0)
%timeit res = op.compute()

2min 45s ± 517 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [ ]:
op = custom_nanpercentile(stack_rechunked, axis = 0)
%timeit res = op.compute()

In [ ]:
raster_percentile = custom_percentile(stack_rechunked, axis = 0).compute()
raster_nan_percentile = custom_nanpercentile(stack_rechunked, axis = 0).compute()

In [ ]:
raster_percentile = custom_percentile(stack_rechunked, axis = 0).compute()

In [ ]:
raster_percentile.shape

In [ ]:
mask = (raster_percentile == numpy.amax(raster_percentile))
mask

In [ ]:
# I guess that percentile is enough

# raster_percentile is read only
raster_percentile_copy = raster_percentile.copy()
# beware the NaNs
raster_percentile_copy[mask] = numpy.nan
ptp, amax = (numpy.nanmax(raster_percentile_copy) - numpy.nanmin(raster_percentile_copy), numpy.nanmax(raster_percentile_copy))
norm_0_1 = (raster_percentile_copy - amax) / -ptp
norm_0_1
plt.figure(figsize = (12, 24))
plt.imshow(norm_0_1, origin='lower', cmap='coolwarm_r')